In [1]:
pip install fairlearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.5/231.5 kB 3.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import export_text
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report as cr
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
import lightgbm as lgb

In [3]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML_project/full_encoded_data_2_no_purpose.csv')

In [4]:
dataset

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,...,encoded_TN,encoded_TX,encoded_UT,encoded_VA,encoded_VT,encoded_WA,encoded_WI,encoded_WV,encoded_WY,Label
0,30000,30000,30000.0,36,22.35,1151.16,4,20,5.0,100000.0,...,0,0,0,0,0,0,0,0,0,0
1,40000,40000,40000.0,60,16.14,975.71,3,14,0.5,45000.0,...,0,0,0,0,0,0,0,0,0,0
2,20000,20000,20000.0,36,7.56,622.68,1,3,10.0,100000.0,...,0,0,0,0,0,1,0,0,0,0
3,4500,4500,4500.0,36,11.31,147.99,2,8,10.0,38500.0,...,0,1,0,0,0,0,0,0,0,0
4,8425,8425,8425.0,36,27.27,345.18,5,25,3.0,450000.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,16000,16000,16000.0,60,11.67,353.25,2,9,10.0,65000.0,...,0,0,0,0,0,0,0,0,0,0
1048571,3500,3500,3500.0,36,24.50,138.24,6,28,9.0,43000.0,...,0,0,0,0,0,0,0,0,0,0
1048572,6275,6275,6275.0,36,14.49,215.97,3,14,3.0,115000.0,...,0,0,0,0,0,0,0,0,0,0
1048573,10375,10375,10375.0,36,16.29,366.25,4,17,3.0,35000.0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
feature_names = dataset.columns.to_list()

In [6]:
feature_names

['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_length',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'recoveries',
 'acc_now_delinq',
 'tot_cur_bal',
 'total_bal_il',
 'all_util',
 'total_cu_tl',
 'avg_cur_bal',
 'bc_util',
 'delinq_amnt',
 'num_sats',
 'pub_rec_bankruptcies',
 'hardship_flag',
 'encoded_ANY',
 'encoded_MORTGAGE',
 'encoded_NONE',
 'encoded_OTHER',
 'encoded_OWN',
 'encoded_RENT',
 'encoded_Not Verified',
 'encoded_Source Verified',
 'encoded_Verified',
 'encoded_Individual',
 'encoded_Joint App',
 'encoded_Cash',
 'encoded_DirectPay',
 'encoded_f',
 'encoded_w',
 'encoded_AK',
 'encoded_AL',
 'encoded_AR',
 'encoded_AZ',
 'encoded_CA',
 'encoded_CO',
 'encoded_CT',
 'encoded_DC',
 'encoded_DE',
 'encoded_FL',
 'encoded_GA',
 'encoded_HI',
 'encoded_IA',
 'encoded_ID',
 'encoded_IL',
 'encoded_IN',
 'encoded_KS',
 'encoded_KY',
 'encoded

In [7]:
features_to_drop = ['encoded_AK','encoded_AL','encoded_AR','encoded_AZ','encoded_CA','encoded_CO','encoded_CT','encoded_DC','encoded_DE','encoded_FL','encoded_GA','encoded_HI','encoded_IA','encoded_ID','encoded_IL','encoded_IN','encoded_KS','encoded_KY','encoded_LA','encoded_MA','encoded_MD','encoded_ME','encoded_MI','encoded_MN','encoded_MO','encoded_MS','encoded_MT','encoded_NC','encoded_ND','encoded_NE','encoded_NH','encoded_NJ','encoded_NM','encoded_NV','encoded_NY','encoded_OH','encoded_OK','encoded_OR','encoded_PA','encoded_RI','encoded_SC','encoded_SD','encoded_TN','encoded_TX','encoded_UT','encoded_VA','encoded_VT','encoded_WA','encoded_WI','encoded_WV','encoded_WY',]
print(len(features_to_drop))

51


We are dropping the features that contain the location information

In [8]:
dataset = dataset.drop(columns = features_to_drop)
dataset

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,...,encoded_Not Verified,encoded_Source Verified,encoded_Verified,encoded_Individual,encoded_Joint App,encoded_Cash,encoded_DirectPay,encoded_f,encoded_w,Label
0,30000,30000,30000.0,36,22.35,1151.16,4,20,5.0,100000.0,...,0,1,0,0,1,1,0,0,1,0
1,40000,40000,40000.0,60,16.14,975.71,3,14,0.5,45000.0,...,0,0,1,0,1,1,0,0,1,0
2,20000,20000,20000.0,36,7.56,622.68,1,3,10.0,100000.0,...,1,0,0,0,1,1,0,0,1,0
3,4500,4500,4500.0,36,11.31,147.99,2,8,10.0,38500.0,...,1,0,0,1,0,1,0,0,1,0
4,8425,8425,8425.0,36,27.27,345.18,5,25,3.0,450000.0,...,0,0,1,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,16000,16000,16000.0,60,11.67,353.25,2,9,10.0,65000.0,...,0,1,0,1,0,1,0,1,0,0
1048571,3500,3500,3500.0,36,24.50,138.24,6,28,9.0,43000.0,...,0,0,1,1,0,1,0,1,0,0
1048572,6275,6275,6275.0,36,14.49,215.97,3,14,3.0,115000.0,...,1,0,0,1,0,1,0,1,0,0
1048573,10375,10375,10375.0,36,16.29,366.25,4,17,3.0,35000.0,...,0,1,0,1,0,1,0,0,1,1


In [9]:
dataset.to_csv('no_state.csv', index=False)


In [10]:
bias_probability = 0.65
default = dataset['Label'] == 1
non_default = dataset['Label'] == 0

default_bias = np.random.choice([0, 1], p=[bias_probability, 1 - bias_probability], size=sum(default))
non_default_bias = np.random.choice([0, 1], p=[1 - bias_probability, bias_probability], size=sum(non_default))


In [11]:
dataset.loc[default, 'sensitive_variable'] = default_bias
dataset.loc[non_default, 'sensitive_variable'] = non_default_bias

In [12]:
dataset

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,...,encoded_Source Verified,encoded_Verified,encoded_Individual,encoded_Joint App,encoded_Cash,encoded_DirectPay,encoded_f,encoded_w,Label,sensitive_variable
0,30000,30000,30000.0,36,22.35,1151.16,4,20,5.0,100000.0,...,1,0,0,1,1,0,0,1,0,1.0
1,40000,40000,40000.0,60,16.14,975.71,3,14,0.5,45000.0,...,0,1,0,1,1,0,0,1,0,1.0
2,20000,20000,20000.0,36,7.56,622.68,1,3,10.0,100000.0,...,0,0,0,1,1,0,0,1,0,1.0
3,4500,4500,4500.0,36,11.31,147.99,2,8,10.0,38500.0,...,0,0,1,0,1,0,0,1,0,1.0
4,8425,8425,8425.0,36,27.27,345.18,5,25,3.0,450000.0,...,0,1,0,1,1,0,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,16000,16000,16000.0,60,11.67,353.25,2,9,10.0,65000.0,...,1,0,1,0,1,0,1,0,0,1.0
1048571,3500,3500,3500.0,36,24.50,138.24,6,28,9.0,43000.0,...,0,1,1,0,1,0,1,0,0,0.0
1048572,6275,6275,6275.0,36,14.49,215.97,3,14,3.0,115000.0,...,0,0,1,0,1,0,1,0,0,1.0
1048573,10375,10375,10375.0,36,16.29,366.25,4,17,3.0,35000.0,...,1,0,1,0,1,0,0,1,1,0.0


In [13]:
x = dataset.drop('Label',axis=1)
y = dataset['Label']

In [14]:
x

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,...,encoded_Not Verified,encoded_Source Verified,encoded_Verified,encoded_Individual,encoded_Joint App,encoded_Cash,encoded_DirectPay,encoded_f,encoded_w,sensitive_variable
0,30000,30000,30000.0,36,22.35,1151.16,4,20,5.0,100000.0,...,0,1,0,0,1,1,0,0,1,1.0
1,40000,40000,40000.0,60,16.14,975.71,3,14,0.5,45000.0,...,0,0,1,0,1,1,0,0,1,1.0
2,20000,20000,20000.0,36,7.56,622.68,1,3,10.0,100000.0,...,1,0,0,0,1,1,0,0,1,1.0
3,4500,4500,4500.0,36,11.31,147.99,2,8,10.0,38500.0,...,1,0,0,1,0,1,0,0,1,1.0
4,8425,8425,8425.0,36,27.27,345.18,5,25,3.0,450000.0,...,0,0,1,0,1,1,0,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,16000,16000,16000.0,60,11.67,353.25,2,9,10.0,65000.0,...,0,1,0,1,0,1,0,1,0,1.0
1048571,3500,3500,3500.0,36,24.50,138.24,6,28,9.0,43000.0,...,0,0,1,1,0,1,0,1,0,0.0
1048572,6275,6275,6275.0,36,14.49,215.97,3,14,3.0,115000.0,...,1,0,0,1,0,1,0,1,0,1.0
1048573,10375,10375,10375.0,36,16.29,366.25,4,17,3.0,35000.0,...,0,1,0,1,0,1,0,0,1,0.0


In [15]:
y

0          0
1          0
2          0
3          0
4          0
          ..
1048570    0
1048571    0
1048572    0
1048573    1
1048574    1
Name: Label, Length: 1048575, dtype: int64

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state = 42)

In [17]:
xgb = XGBClassifier(booster='gbtree',
    learning_rate=0.1,
    max_depth=20,
    subsample=0.8,
    colsample_bytree=0.8,
    n_estimators=200,
    random_state=78
)

In [18]:
xgb.fit(x_train,y_train)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=20, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=78, ...)

In [19]:
y_pred = xgb.predict(x_test)

In [20]:
len(y_pred)

314573

In [21]:
len(y_test)

314573

In [22]:
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9365012254707175


In [23]:
feature_imp = xgb.feature_importances_
feature_imp_df = pd.DataFrame({'Feature': x.columns, 'Importance': feature_imp})
feature_imp_df = feature_imp_df.sort_values(by='Importance', ascending=False)

print("Feature Importance:")
print(feature_imp_df)

Feature Importance:
                    Feature  Importance
17               recoveries    0.991169
44       sensitive_variable    0.003495
3                      term    0.000476
6                     grade    0.000352
7                 sub_grade    0.000319
34             encoded_RENT    0.000144
21                 all_util    0.000141
39        encoded_Joint App    0.000138
43                encoded_w    0.000137
40             encoded_Cash    0.000132
42                encoded_f    0.000131
38       encoded_Individual    0.000128
18           acc_now_delinq    0.000124
23              avg_cur_bal    0.000123
25              delinq_amnt    0.000123
20             total_bal_il    0.000120
13                  pub_rec    0.000119
2           funded_amnt_inv    0.000114
4                  int_rate    0.000114
14                revol_bal    0.000113
11              delinq_2yrs    0.000111
30         encoded_MORTGAGE    0.000111
5               installment    0.000110
16                to

In [24]:
from fairlearn.metrics import (
    demographic_parity_difference,
    equalized_odds_difference,
    MetricFrame,
    true_positive_rate,
    false_positive_rate
)

In [25]:
# Assuming y_pred are your model predictions and y_true are true labels
metric_frame = MetricFrame(
    metrics={
        'true_positive_rate': true_positive_rate,
        'false_positive_rate': false_positive_rate
    },
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=x_test['sensitive_variable']
)

print("Demographic Parity Difference:", demographic_parity_difference(y_test, y_pred, sensitive_features=x_test['sensitive_variable']))
print("Equalized Odds Difference:", equalized_odds_difference(y_test, y_pred, sensitive_features=x_test['sensitive_variable']))
print("Metric Frame:\n", metric_frame.by_group)

Demographic Parity Difference: 0.14634187815793967
Equalized Odds Difference: 0.018157538518301597
Metric Frame:
                     true_positive_rate  false_positive_rate
sensitive_variable                                         
0.0                           0.698111             0.008755
1.0                           0.679954             0.000073


The results from your fairness evaluation can be interpreted as follows:

Demographic Parity Difference (0.1463): This value indicates the difference in the rate of positive predictions between the two groups (0 and 1 in your bias_variable). A value of 0.1463 suggests a moderate disparity, with one group receiving positive outcomes at a rate approximately 14.63% higher than the other. This can imply a potential bias in how the model treats different groups.

Equalized Odds Difference (0.0182): This metric measures the difference in both false positive rates (FPR) and true positive rates (TPR) between groups. A lower value, like 0.0182, indicates that both TPR and FPR are relatively similar across groups, suggesting that the model is more fair in terms of both false alarms and correctly identified positive cases.

Metric Frame:

True Positive Rate (TPR): Group 0 has a TPR of 0.6981, and Group 1 has a TPR of 0.6799. These values are relatively close, indicating that both groups have a similar likelihood of being correctly identified as positive by the model.
False Positive Rate (FPR): Group 0 has an FPR of 0.0088, while Group 1 has a significantly lower FPR of 0.0001. This suggests that Group 0 is more likely to experience false alarms compared to Group 1.
Overall, your model shows some disparity in terms of demographic parity but performs relatively well in terms of equalized odds. The noticeable difference in FPR between the groups might need further investigation and potential mitigation.